# Processing California's public payrolls

This notebook processess annual government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [1]:
import pandas as pd
import zipfile
from urllib.request import urlopen 
import pyarrow
import cpi
import os
import glob
import io
import requests
import matplotlib
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Download zipped salary tables by year and agency type

In [2]:
os.chdir('/Users/mhustiles/data/data/controller/input/')

In [3]:
formaturl = lambda x: 'https://publicpay.ca.gov/RawExport/' + f'{x[1]}_' + f'{x[0]}' + '.zip'

In [4]:
# metadata = []
# for y in range(2009,2019):
#     for e in ['City', 'County', 'SpecialDistrict', 'StateDepartment']:
# #     for e in ['City', 'County']:
#         metadata.append(dict(entity = e, year = y, url = formaturl((e, y))))

### Extract CSVs from .zip files, and then discard the .zip files

In [5]:
# for m in metadata:
#     !wget '{m['url']}'
#     !unzip \*.zip
#     !rm -f *.zip

---

### Read all the text files, loop and store them in a dataframe

In [6]:
path = '/Users/mhustiles/data/data/controller/input/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, encoding = "ISO-8859-1", low_memory=False, dtype = {'DepartmentOrSubdivision': 'object', 'Year': 'object'}) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

In [7]:
concatenated_df.head()

Year EmployerType EmployerName DepartmentOrSubdivision  \
0  2010       County      Alameda  Ac Central Collections   
1  2010       County      Alameda  Ac Central Collections   
2  2010       County      Alameda  Ac Central Collections   
3  2010       County      Alameda  Ac Central Collections   
4  2010       County      Alameda  Ac Central Collections   

                Position  ElectedOfficial  Judicial OtherPositions  \
0     Accountant-Auditor              nan       nan            NaN   
1  Accounting Technician              nan       nan            NaN   
2    Admin Specialist II              nan       nan            NaN   
3    Auditor Associate I              nan       nan            NaN   
4    Auditor Associate I              nan       nan            NaN   

   MinPositionSalary  MaxPositionSalary  ...  HealthDentalVision  \
0             53,566             58,324  ...               1,088   
1             45,427             55,224  ...              12,375   
2             56,451             75,608  ...               4,718   
3             40,033             47,346  ...               8,710   
4             40,033             47,346  ...               4,916   

   TotalRetirementAndHealthContribution  PensionFormula        EmployerURL  \
0                                 1,088           2%@62  www.acgov.org/hrs   
1                                14,031           2%@57  www.acgov.org/hrs   
2                                 5,486           2%@57  www.acgov.org/hrs   
3                                 8,710           2%@62  www.acgov.org/hrs   
4                                 4,916           2%@62  www.acgov.org/hrs   

   EmployerPopulation  LastUpdatedDate  EmployerCounty  \
0           1,517,756       06/06/2016         Alameda   
1           1,517,756       06/06/2016         Alameda   
2           1,517,756       06/06/2016         Alameda   
3           1,517,756       06/06/2016         Alameda   
4           1,517,756       06/06/2016         Alameda   

   SpecialDistrictActivities  IncludesUnfundedLiability  SpecialDistrictType  
0                        NaN                        nan                  NaN  
1                        NaN                        nan                  NaN  
2                        NaN                        nan                  NaN  
3                        NaN                        nan                  NaN  
4                        NaN                        nan                  NaN  

[5 rows x 29 columns]

### Trim the dataframe to the columns we need

In [8]:
payroll = \
pd.DataFrame(concatenated_df[['Year','EmployerType','EmployerPopulation','EmployerName','DepartmentOrSubdivision',\
                 'Position','OvertimePay','TotalWages', 'TotalRetirementAndHealthContribution', 'EmployerCounty']])

In [9]:
payroll.head(1)

Year EmployerType  EmployerPopulation EmployerName DepartmentOrSubdivision  \
0  2010       County           1,517,756      Alameda  Ac Central Collections   

             Position  OvertimePay  TotalWages  \
0  Accountant-Auditor          nan       4,120   

   TotalRetirementAndHealthContribution EmployerCounty  
0                                 1,088        Alameda

### Not everyone reports (or pays) overtime

In [10]:
payroll['OvertimePay'].fillna(0, inplace = True)
payroll['DepartmentOrSubdivision'].fillna('NOT LISTED', inplace = True)
payroll['EmployerPopulation'].fillna(0, inplace = True)

In [11]:
payroll.head(1)

Year EmployerType  EmployerPopulation EmployerName DepartmentOrSubdivision  \
0  2010       County           1,517,756      Alameda  Ac Central Collections   

             Position  OvertimePay  TotalWages  \
0  Accountant-Auditor            0       4,120   

   TotalRetirementAndHealthContribution EmployerCounty  
0                                 1,088        Alameda

### Clean up column headers

In [12]:
payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [13]:
payroll.rename(columns = {
'employertype':'type',
'employerpopulation':'population',
'employername':'employer',
'departmentorsubdivision':'department',
'overtimepay':'overtime',
'totalretirementandhealthcontribution':'benefits',
'totalwages':'wages',
 }, inplace = True)

### Uppercase everything because their title casing across hundreds of agencies is janky

In [14]:
payroll = payroll.apply(lambda x: x.astype(str).str.upper())

### Convert year column to datetime year for CPI function

In [15]:
payroll['year'] = pd.to_datetime(payroll['year'])

In [16]:
payroll['year'] = payroll['year'].dt.year

### Adjust overtime on each row for CPI according to its respective year

In [17]:
payroll[['overtime', 'population', 'wages', 'benefits']] = \
payroll[['overtime', 'population', 'wages', 'benefits']].astype(float)

In [18]:
payroll['adjusted_overtime'] = payroll.apply(lambda x: cpi.inflate(x.overtime, x.year), axis=1)

In [19]:
payroll['adjusted_wages'] = payroll.apply(lambda x: cpi.inflate(x.wages, x.year), axis=1)

In [20]:
payroll['adjusted_benefits'] = payroll.apply(lambda x: cpi.inflate(x.benefits, x.year), axis=1)

### How does the dataframe look? 

In [21]:
payroll.head()

year    type  population employer              department  \
0  2010  COUNTY   1,517,756  ALAMEDA  AC CENTRAL COLLECTIONS   
1  2010  COUNTY   1,517,756  ALAMEDA  AC CENTRAL COLLECTIONS   
2  2010  COUNTY   1,517,756  ALAMEDA  AC CENTRAL COLLECTIONS   
3  2010  COUNTY   1,517,756  ALAMEDA  AC CENTRAL COLLECTIONS   
4  2010  COUNTY   1,517,756  ALAMEDA  AC CENTRAL COLLECTIONS   

                position  overtime  wages  benefits employercounty  \
0     ACCOUNTANT-AUDITOR         0  4,120     1,088        ALAMEDA   
1  ACCOUNTING TECHNICIAN         0 54,438    14,031        ALAMEDA   
2    ADMIN SPECIALIST II         0 25,376     5,486        ALAMEDA   
3    AUDITOR ASSOCIATE I         0 31,225     8,710        ALAMEDA   
4    AUDITOR ASSOCIATE I         0 27,677     4,916        ALAMEDA   

   adjusted_overtime  adjusted_wages  adjusted_benefits  
0                  0           4,744              1,253  
1                  0          62,689             16,158  
2                  0          29,222              6,318  
3                  0          35,958             10,030  
4                  0          31,872              5,661

### How many records do we have here?

In [22]:
# How many records?
len(payroll)

10826479

### Clean up some employer names

In [1]:
payroll['employer']\
    .replace('SACRAMENTO METROPOLITAN FIRE DISTRICT', 'SACRAMENTO METRO FIRE', inplace=True)

NameError: name 'payroll' is not defined

In [24]:
payroll['employer']\
    .replace('FORESTRY & FIRE PROTECTION, CALIFORNIA DEPARTMENT OF', 'CALFIRE', inplace=True)

In [ ]:
payroll['type']\
    .replace('STATE DEPARTMENT', 'STATE', inplace=True)

In [25]:
payroll['employer']\
    .replace('ORANGE COUNTY FIRE AUTHORITY', 'OC FIRE AUTHORITY', inplace=True)

### Export table of all titles to help ID fire roles

In [26]:
positions_all = payroll.groupby(['position', 'employer', 'type', 'department'])\
    .agg('size').reset_index()

In [27]:
positions_all['position'] = positions_all['position']\
                            .str.replace('!ST ', '', regex=False)\
                            .str.replace('1ST ', '', regex=False)\
                            .str.replace('2ND', '', regex=False)\
                            .str.replace('.E.', '', regex=False)\
                            .str.replace(' - ', '', regex=False)\
                            .str.replace('\d', '')\
                            .str.replace('[.!?\\-]', '')

In [28]:
positions_fire = positions_all[ (positions_all['position'].str.contains('FIRE')) |\
                                       (positions_all['type'].str.contains('FIRE')) |\
                                       (positions_all['department'].str.contains('FIRE')) |\
                                       (positions_all['employer'].str.contains('FIRE'))]

In [29]:
positions_fire_grouped = positions_fire.groupby(['position']).agg('size')

In [33]:
positions_fire_grouped.to_csv('/Users/mhustiles/data/github/notebooks/payroll/input/positions_fire_grouped.csv', header=True)

---

### Export to a lightweight format

In [34]:
payroll.reset_index().to_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [35]:
payroll.reset_index().to_csv('/Users/mhustiles/data/data/controller/output/payroll.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx